# Running inference

In [1]:
import librosa as lr
import octafilt3r.filter as o3f
import octafilt3r.weightings as o3w
import os
from tensorflow.keras.models import load_model
import numpy as np
import pipeline.config as conf
import pipeline.scale as scale

In [2]:
param_dict = conf.open_params()

class_of_interest = "sweep"
base_path = "Datasets/sine_data/" + class_of_interest + "/"
stop_at = 30
i = 0
true = 0
false = 0

N4CED = load_model(param_dict["CNN_name"])

for file in os.listdir(base_path):

    if(i == stop_at):
        break

    """
    LOAD DATA
    """
    wav, fs = lr.load(base_path + file, sr=param_dict['fs'], duration=param_dict["max_dur"])
    wav = o3w.A_weight(wav, fs)


    if len(wav) < fs:
        continue


    """
    EXTRACT FEATURES
    """
    feats, fcs, leqs = o3f.rolling_oct_bank(
        wav,
        fs,
        param_dict['oct_bw_ratio'],
        param_dict['order'],
        param_dict['fmax'],
        param_dict['fmin'],
        param_dict['frame_size'],
        param_dict['dec_stages'],
        param_dict['dec_ord']
    )


    """
    RESHAPE FEATURES
    """
    df = scale.feats2frames(feats, param_dict['frame_size'], fs)


    """
    SCALE FEATURES
    """
    df = scale.normalize_dbfs(df, leqs)
    df = df[..., np.newaxis]


    """
    RUN MODEL
    """
    p = N4CED.predict(df)


    """
    PROCESS RESULTS
    """
    predicted = param_dict["class_map"][np.argmax(np.mean(p, axis=0))]
    print(f'Predicted class: "{predicted}" with {int(100 * (p[0][np.argmax(p, 1)][0]))} score.')
    if predicted == class_of_interest:
        true += 1
    else:
        false += 1
    i += 1

print('----- done. -----')
print(f'{true} true predictions\n{false} false predictions')
print(f'test accuracy: {int((true/i)*100)}%\nerror ratio: {int((false/i)*100)}%')

Predicted class: "sweep" with 66 score.
Predicted class: "sweep" with 64 score.
Predicted class: "sweep" with 97 score.
Predicted class: "sweep" with 67 score.
Predicted class: "sweep" with 69 score.
Predicted class: "sweep" with 98 score.
Predicted class: "sweep" with 42 score.
Predicted class: "sweep" with 40 score.
Predicted class: "sweep" with 96 score.
Predicted class: "sweep" with 78 score.
Predicted class: "sweep" with 70 score.
Predicted class: "sweep" with 97 score.
Predicted class: "sweep" with 87 score.
Predicted class: "sweep" with 78 score.
Predicted class: "sweep" with 96 score.
Predicted class: "sweep" with 54 score.
Predicted class: "sweep" with 86 score.
Predicted class: "sweep" with 98 score.
Predicted class: "sweep" with 72 score.
Predicted class: "sweep" with 89 score.
Predicted class: "sweep" with 93 score.
Predicted class: "sweep" with 79 score.
Predicted class: "sweep" with 68 score.
Predicted class: "sweep" with 98 score.
Predicted class: "sweep" with 79 score.
